In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
import optuna
import logging
import json
import os
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from category_encoders import TargetEncoder

# Configure logging
os.makedirs("..data", exist_ok=True)
logging.basicConfig(filename='../data/log_file.log', level=logging.INFO)

In [2]:
# Load training data
df = pd.read_csv("../data/train.csv")

# Drop unnecessary columns
df.drop(["PassengerId", "Name"], axis=1, inplace=True, errors="ignore")

# Define numerical and categorical columns
numerical_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

# Handle missing values
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].mean())
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Feature Engineering: Split "Cabin" into "Deck", "CabinNumber", "Side"
df[["Deck", "CabinNumber", "Side"]] = df["Cabin"].str.split("/", expand=True)
df.drop("Cabin", axis=1, inplace=True)
df["CabinNumber"] = pd.to_numeric(df["CabinNumber"], errors="coerce").fillna(0)

# Update categorical columns list
categorical_cols.extend(["Deck", "Side"])

# Target Encoding
target_encoding_mappings = {col: df.groupby(col)["Transported"].mean() for col in categorical_cols}
for col in categorical_cols:
    df[f"{col}_encoded"] = df[col].map(target_encoding_mappings[col])
df.drop(categorical_cols, axis=1, inplace=True)

# Feature Engineering
df["TotalSpending"] = df[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)
df["Age_Spending"] = df["Age"] * df["TotalSpending"]
df["High_Cabin"] = (df["CabinNumber"] > df["CabinNumber"].median()).astype(int)
df.drop("CabinNumber", axis=1, inplace=True)

# Log Transformations for Spending Columns
spending_cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
for col in spending_cols:
    df[f"{col}_log"] = np.log1p(df[col])
df.drop(spending_cols, axis=1, inplace=True)

# Train-Test Split
X = df.drop("Transported", axis=1)
y = df["Transported"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Data Preprocessing Completed!")

Training Data Preprocessing Completed!


In [3]:
def objective(trial):
    # Hyperparameters to tune
    params = {
        "iterations": trial.suggest_int("iterations", 200, 1000),
        "depth": trial.suggest_int("depth", 4, 9),
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.2, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 5, log=True),
        "border_count": trial.suggest_int("border_count", 64, 255),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 5, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "loss_function": "Logloss",
        "verbose": False,
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 10, 50),  # Early stopping to prevent overfitting
    }

    # Prepare the training pool for CatBoost
    train_pool = Pool(X_train, label=y_train)

    # Run cross-validation with early stopping and all available threads
    cv_results = cv(
        params=params,
        pool=train_pool,
        fold_count=5,  # 5-fold cross-validation
        shuffle=True,
        partition_random_seed=42,
        stratified=True,
        early_stopping_rounds=params["early_stopping_rounds"],
        verbose_eval=False,
    )

    # Extract the best test metric (Logloss)
    best_metric = [col for col in cv_results.columns if "test" in col and "mean" in col][0]  # Select the correct column
    return cv_results[best_metric].min()  # We want to minimize Logloss, so return the minimum value

logging.info("Starting Optuna optimization...")
# Run Optuna with parallel trials
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, n_jobs=-1)
best_params = study.best_params
with open("../mlops_hw1/best_params.json", "w") as f:
    json.dump(best_params, f)

logging.info(f"Optimization complete. Best Parameters: {best_params}")
print("Best Parameters:", study.best_params)

[I 2025-03-20 17:13:08,043] A new study created in memory with name: no-name-5d2ee80d-56cd-456d-a69b-11515f84fa07


Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]
Training on fold [0/5]

bestTest = 0.4085607518
bestIteration = 32

Training on fold [1/5]

bestTest = 0.4014642329
bestIteration = 64

Training on fold [1/5]

bestTest = 0.4084762782
bestIteration = 97

Training on fold [1/5]

bestTest = 0.407055551
bestIteration = 96


bestTest = 0.4051441918
bestIteration = 134

Training on fold [1/5]
Training on fold [1/5]

bestTest = 0.4193942336
bestIteration = 36

Training on fold [2/5]

bestTest = 0.4045947748
bestIteration = 136

Training on fold [1/5]

bestTest = 0.4125388192
bestIteration = 60

Training on fold [1/5]

bestTest = 0.4105509378
bestIteration = 73

Training on fold [2/5]

bestTest = 0.38169

[I 2025-03-20 17:13:18,596] Trial 1 finished with value: 0.4123901523621859 and parameters: {'iterations': 700, 'depth': 7, 'learning_rate': 0.14812157065843756, 'l2_leaf_reg': 0.06715653808550483, 'border_count': 229, 'random_strength': 0.05301032495033394, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree', 'early_stopping_rounds': 27}. Best is trial 1 with value: 0.4123901523621859.



bestTest = 0.4251995016
bestIteration = 48

Training on fold [0/5]

bestTest = 0.3660728572
bestIteration = 130

Training on fold [3/5]

bestTest = 0.4094503536
bestIteration = 123

Training on fold [2/5]

bestTest = 0.4067061105
bestIteration = 77

Training on fold [4/5]

bestTest = 0.3679129703
bestIteration = 176


bestTest = 0.3804859453
bestIteration = 112

Training on fold [3/5]
Training on fold [3/5]

bestTest = 0.4256060906
bestIteration = 111

Training on fold [2/5]

bestTest = 0.4058628461
bestIteration = 73

Training on fold [4/5]

bestTest = 0.404383052
bestIteration = 211

Training on fold [1/5]


[I 2025-03-20 17:13:21,674] Trial 9 finished with value: 0.40258535505719617 and parameters: {'iterations': 425, 'depth': 4, 'learning_rate': 0.14193644532462843, 'l2_leaf_reg': 3.126945377787767, 'border_count': 77, 'random_strength': 0.06128562873814301, 'bootstrap_type': 'MVS', 'grow_policy': 'Lossguide', 'early_stopping_rounds': 17}. Best is trial 9 with value: 0.40258535505719617.


...


bestTest = 0.3952615989
bestIteration = 143

Training on fold [4/5]

bestTest = 0.4156809585
bestIteration = 446


bestTest = 0.3958940524
bestIteration = 161

Training on fold [1/5]

bestTest = 0.3637232531
bestIteration = 132

Training on fold [3/5]

bestTest = 0.3953270837
bestIteration = 146

Training on fold [4/5]


[I 2025-03-20 17:14:49,776] Trial 0 finished with value: 0.4012770572789268 and parameters: {'iterations': 495, 'depth': 5, 'learning_rate': 0.02032469933278671, 'l2_leaf_reg': 0.6052031073501143, 'border_count': 211, 'random_strength': 1.9248966215245957, 'bootstrap_type': 'MVS', 'grow_policy': 'Lossguide', 'early_stopping_rounds': 22}. Best is trial 24 with value: 0.397811063928694.



bestTest = 0.4185231569
bestIteration = 494



[I 2025-03-20 17:14:53,211] Trial 39 finished with value: 0.40066458949064165 and parameters: {'iterations': 462, 'depth': 5, 'learning_rate': 0.09534466510724621, 'l2_leaf_reg': 0.4480029634338219, 'border_count': 130, 'random_strength': 0.5913190762763267, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'early_stopping_rounds': 18}. Best is trial 24 with value: 0.397811063928694.
[I 2025-03-20 17:14:54,795] Trial 40 finished with value: 0.400139189647082 and parameters: {'iterations': 474, 'depth': 5, 'learning_rate': 0.09234479645216151, 'l2_leaf_reg': 0.7263638844493493, 'border_count': 141, 'random_strength': 0.5369954105637048, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'early_stopping_rounds': 19}. Best is trial 24 with value: 0.397811063928694.
[I 2025-03-20 17:14:54,920] Trial 12 finished with value: 0.4100780707937576 and parameters: {'iterations': 329, 'depth': 9, 'learning_rate': 0.03600951182280689, 'l2_leaf_reg': 0.04682936724714622, 'border_count

Best Parameters: {'iterations': 657, 'depth': 7, 'learning_rate': 0.055239378192584745, 'l2_leaf_reg': 1.0018746025288165, 'border_count': 113, 'random_strength': 1.413719482197632, 'bootstrap_type': 'MVS', 'grow_policy': 'Lossguide', 'early_stopping_rounds': 16}


In [4]:
final_model = CatBoostClassifier(**study.best_params, verbose=0)
final_model.fit(X_train, y_train)
print("Final Model Accuracy:", accuracy_score(y_val, final_model.predict(X_val)))
final_model.save_model("../model/final_model.cbm")

Final Model Accuracy: 0.8016101207590569


In [5]:
# Load test data
df_test = pd.read_csv("../data/test.csv")
passenger_ids = df_test["PassengerId"]
df_test.drop(["PassengerId", "Name"], axis=1, inplace=True, errors="ignore")

# Handle missing numerical values
for col in numerical_cols:
    df_test[col] = df_test[col].fillna(df_test[col].mean())

# Feature Engineering: Split "Cabin" into "Deck", "CabinNumber", "Side"
if "Cabin" in df_test.columns:
    df_test[["Deck", "CabinNumber", "Side"]] = df_test["Cabin"].str.split("/", expand=True)
    df_test.drop("Cabin", axis=1, inplace=True)
else:
    df_test["Deck"], df_test["CabinNumber"], df_test["Side"] = "Unknown", 0, "Unknown"

df_test["CabinNumber"] = pd.to_numeric(df_test["CabinNumber"], errors="coerce").fillna(0)

# Ensure all categorical columns exist
for col in categorical_cols:
    if col not in df_test.columns:
        df_test[col] = "Unknown"

# Apply target encoding using saved mappings (with fallback for unseen categories)
for col in categorical_cols:
    df_test[f"{col}_encoded"] = df_test[col].map(target_encoding_mappings[col]).fillna(y.mean())

df_test.drop(categorical_cols, axis=1, inplace=True)

# Feature Engineering
df_test["TotalSpending"] = df_test[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].sum(axis=1)
df_test["Age_Spending"] = df_test["Age"] * df_test["TotalSpending"]

# Ensure you calculate "High_Cabin" before dropping "CabinNumber"
df_test["High_Cabin"] = (df_test["CabinNumber"] > df_test["CabinNumber"].median()).astype(int)

# Drop "CabinNumber" after creating "High_Cabin"
df_test.drop("CabinNumber", axis=1, inplace=True)


# Log Transformations
for col in spending_cols:
    df_test[f"{col}_log"] = np.log1p(df_test[col])
df_test.drop(spending_cols, axis=1, inplace=True)

# Ensure test data matches training data columns
df_test = df_test.reindex(columns=X_train.columns, fill_value=0)

print("Test Data Preprocessing Completed!")

Test Data Preprocessing Completed!


In [6]:
final_model = CatBoostClassifier()
final_model.load_model("../model/final_model.cbm")
predictions = final_model.predict(df_test)
submission = pd.DataFrame({"PassengerId": passenger_ids, "Transported": predictions})
submission.to_csv("../data/submission.csv", index=False)
print("Submission saved!")

Submission saved!
